In [1]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd
import re
import math
import numpy as np
import time

In [2]:
startnum = 1

In [3]:
# アクセスURL
url = 'http://fishpix.kahaku.go.jp/fishimage/search?START=%d&JPN_FAMILY=&FAMILY=&JPN_NAME=&SPECIES=&LOCALITY=&FISH_Y=&FISH_M=&FISH_D=&PERSON=&PHOTO_ID=&JPN_FAMILY_OPT=0&FAMILY_OPT=0&JPN_NAME_OPT=0&SPECIES_OPT=0&LOCALITY_OPT=1&PERSON_OPT=1&PHOTO_ID_OPT=1'%startnum

In [4]:
def url_generate(url):
#     if startnum == 1:
    response = urllib.request.urlopen(url)
    soup = BeautifulSoup(response, "html.parser")
    text = soup.find('span', class_= 'captionTimes').text
    num = text.split("件")[0].split()[0]
    print('%s images were found'%num)
    page = math.ceil(int(num)/20.)
    page_nums = list(np.arange(1, page*20, 20))
    return page_nums

In [5]:
page_nums = url_generate(url)

109196 images were found


In [6]:
def information_getter(url):
    '''
    urlを渡すと情報を返す
    '''
    
    response = urllib.request.urlopen(url)
    soup = BeautifulSoup(response, "html.parser")
    
    result = soup.findAll('span', class_= 'result')
    names = []
    for x in result:
        names.append(x.text)
    result_ = soup.findAll('span', class_= 'resultHelvetica')
    namesHel = []
    for x in result_:
        namesHel.append(x.text)
    
    caption = soup.findAll('span', class_= 'captionTimes')
    caps = []
    for x in caption:
        cap = x.find('a',href=re.compile('^detail'))
        if cap != None:
            caps.append(cap.text)
    
    imgsoup = soup.findAll("img")
    img_path = []
    for x in imgsoup:
        if x['height'] == "130":
            img_path.append(x['src'])

    return names, namesHel, caps, img_path

In [7]:
def dataframe_maker(page_nums):
    df = pd.DataFrame(columns = ['name', 'nameEng', 'caption', 'URL'])
    
    for n in page_nums:
        url = 'http://fishpix.kahaku.go.jp/fishimage/search?START=%d&JPN_FAMILY=&FAMILY=&JPN_NAME=&SPECIES=&LOCALITY=&FISH_Y=&FISH_M=&FISH_D=&PERSON=&PHOTO_ID=&JPN_FAMILY_OPT=0&FAMILY_OPT=0&JPN_NAME_OPT=0&SPECIES_OPT=0&LOCALITY_OPT=1&PERSON_OPT=1&PHOTO_ID_OPT=1'%n
        names, namesHel, caps, img_url = information_getter(url)
        df_batch = pd.DataFrame([names, namesHel, caps, img_url],\
                                index=['name', 'nameEng', 'caption', 'URL']).transpose()
#         display(df_batch.head())
        df = pd.concat([df, df_batch])
#         display(df.tail())
        print(n)
#         display(names)
#         display(caps)
        time.sleep(2)
    return df

In [ ]:
df = dataframe_maker(page_nums)

1
21
41
61
81
101
121
141
161
181
201
221
241
261
281
301
321
341
361
381
401
421
441
461
481
501
521
541
561
581
601
621
641
661
681
701
721
741
761
781
801
821
841
861
881
901
921
941
961
981
1001
1021
1041
1061
1081
1101
1121
1141
1161
1181
1201
1221
1241
1261
1281
1301
1321
1341
1361
1381
1401
1421
1441
1461
1481
1501
1521
1541
1561
1581
1601
1621
1641
1661
1681
1701
1721
1741
1761
1781
1801
1821
1841
1861
1881
1901
1921
1941
1961
1981
2001
2021
2041
2061
2081
2101
2121
2141
2161
2181
2201
2221
2241
2261
2281
2301
2321
2341
2361
2381
2401
2421
2441
2461
2481
2501
2521
2541
2561
2581
2601
2621
2641
2661
2681
2701
2721
2741
2761
2781
2801
2821
2841
2861
2881
2901
2921
2941
2961
2981
3001
3021
3041
3061
3081
3101
3121
3141
3161
3181
3201
3221
3241
3261
3281
3301
3321
3341
3361
3381
3401
3421
3441
3461
3481
3501
3521
3541
3561
3581
3601
3621
3641
3661
3681
3701
3721
3741
3761
3781
3801
3821
3841
3861
3881
3901
3921
3941
3961
3981
4001
4021
4041
4061
4081
4101
4121
4141
4161
4181
4201
4

In [10]:
df.count()

name       2832
nameEng    2832
caption    2832
URL        2832
dtype: int64

In [13]:
df['name'].value_counts()

トゲチョウチョウウオ                            159
ムレハタタテダイ                              140
チョウチョウウオ                              136
フウライチョウチョウウオ                          122
チョウハン                                 106
ミゾレチョウチョウウオ                            99
シラコダイ                                  93
アケボノチョウチョウウオ                           88
フエヤッコダイ                                86
ハタタテダイ                                 85
ミナミハタタテダイ                              78
ミスジチョウチョウウオ                            74
タキゲンロクダイ                               68
ヤリカタギ                                  68
ユウゼン                                   59
セグロチョウチョウウオ                            56
シチセンチョウチョウウオ                           55
ゴマチョウチョウウオ                             53
ゲンロクダイ                                 53
ウミヅキチョウチョウウオ                           52
イッテンチョウチョウウオ                           50
カガミチョウチョウウオ                            50
ハナグロチョウチョウウオ                           48
スミツキトノサマダイ                        

In [14]:
df.to_csv("butterfly.csv")